In [3]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GaussianNoise
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras.utils import vis_utils as vizu
from keras.layers import Dropout
from keras import callbacks
from keras import backend
from scipy.integrate import simps
from scipy.stats import f
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, mutual_info_regression
def knn_baseline_model():
    
    def root_mean_squared_error(y_true, y_pred):
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    # create model
    model = Sequential()
    #model.add(GaussianNoise(stddev=0.1, input_shape=(14,)))
    model.add(Dense(7, activation='relu', input_shape=(16,)))
    model.add(Dropout(0.5))
    model.add(GaussianNoise(0.1))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss=root_mean_squared_error, optimizer='adam')
    #model.compile(loss='mse', optimizer='adam')
    return model
def coeff_determination(y_true, y_pred):
    ssres = np.sum(np.square(y_true - y_pred))
    sstot = np.sum(np.square(y_true - np.mean(y_pred)))
    return 1 - ssres / sstot
def REC(y_true , y_pred):
    
    Accuracy = []
    Begin_Range = 0
    End_Range = 1.5
    Interval_Size = 0.01
    
    # List of epsilons
    Epsilon = np.arange(Begin_Range , End_Range , Interval_Size)
    
    # Main Loops
    for i in range(len(Epsilon)):
        count = 0.0
        for j in range(len(y_true)):
            y_true[j]=float(y_true[j])
            y_true[j]
            y_pred[j]
            
            np.linalg.norm(y_true[j] - y_pred[j])
            np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]
            
            if np.linalg.norm(y_true[j] - y_pred[j]) / np.sqrt( np.linalg.norm(y_true[j]) **2 + np.linalg.norm(y_pred[j])**2 ) < Epsilon[i]:
                count = count + 1
        
        Accuracy.append(count/len(y_true))
    
    # Calculating Area Under Curve using Simpson's rule
    AUC = simps(Accuracy , Epsilon ) / End_Range
        
    # returning epsilon , accuracy , area under curve    
    return Epsilon, Accuracy, AUC
def model_evaluate (model, X_train, X_test, y_train, y_test):

    y_pred = model.predict(X_test)
    Deviation, Accuracy, auc = REC(y_test.values, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    #r2 = r2_score(y_test, y_pred)
    r2 = coeff_determination(y_test, y_pred)
    mape = ((y_test - y_pred) / y_test).abs().mean()*100
    
    f_test = np.var(y_test) / np.var(y_pred)
    df1 = len(y_test) - 1
    df2 = len(y_pred) - 1
    p_value = 1 - 2 * abs(0.5 - f.cdf(f_test, df1, df2))
    
    p_stars = ''
    if p_value <= 0.05:
        p_stars = '*'
    if p_value <= 0.01:
        p_stars = '**'
    if p_value <= 0.001:
        p_stars = '***'
    #print(F, p_value)
    
    y_validate = model.predict(X_train)
    training_loss = sqrt(mean_squared_error(y_train, y_validate))
    
    return_dict = {'rmse':rmse, 
                   'mape':mape, 
                   'r2':r2, 
                   'auc':auc,
                   'training_loss':training_loss, 
                   'f_test':f_test,
                   'p':p_stars,
                   'y_pred':y_pred, 
                   'y_validate':y_validate}
    
    return return_dict

In [5]:
os1052 = pd.read_csv('1052OS_w10.csv')
os1052.fillna(0.0, inplace=True)
os1052 = os1052.drop(['name'], axis=1)
os1052 = os1052.drop(['username'], axis=1)
os1052 = os1052.drop(['final_score'], axis=1)
os1052 = os1052.drop(['label_2'], axis=1)
os1052.head(3)

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,4,5,325,0,0,2,67,0,0,0,28,1,0,100,0,60,12
1,2,3,1,2,18,2,37,0,0,0,28,0,471,470,0,167,70
2,5,30,780,3,19,23,114,0,0,0,28,22,486,380,0,114,81


In [7]:
data=os1052
data.to_csv('cycu os1052.csv')
data_y= data['adjust_score']
data_x = data.drop(['adjust_score'], axis=1)

In [8]:
data

,video_watching_days,video_watching_times,video_watching_total_time,bbs_days,bbs_num,video_watching_num,video_time_between_start_first_watching,pre_watching_num,in_watching_num,re_watching_num,no_watching_num,post_watching_num,online_quiz,online_hw,pgm,quiz,adjust_score
0,4,5,325,0,0,2,67,0,0,0,28,1,0,100,0,60,12
1,2,3,1,2,18,2,37,0,0,0,28,0,471,470,0,167,70
2,5,30,780,3,19,23,114,0,0,0,28,22,486,380,0,114,81
3,3,3,0,0,0,0,0,0,0,0,28,0,0,0,0,85,20
4,9,15,55,4,17,6,57,0,1,0,27,3,400,420,0,134,71
5,1,1,0,2,19,0,0,0,0,0,28,0,500,390,0,128,45
6,1,1,0,0,0,0,0,0,0,0,28,0,0,0,0,25,9
7,3,8,115,0,0,2,96,0,0,0,28,1,100,0,0,50,15
8,8,25,7,0,0,10,117,0,0,0,28,5,386,60,0,105,61
9,7,18,343,0,0,7,94,0,0,0,28,5,471,440,0,105,81


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3)

In [10]:
mlp = KerasRegressor(build_fn=knn_baseline_model, epochs=1000, batch_size=2, verbose=0)

In [11]:
mlp.fit(X_train, y_train) 
result_dict = model_evaluate (mlp, X_train, X_test, y_train, y_test)
print(result_dict)

{'rmse': 29.328815787554095, 'mape': 51.333772572749126, 'r2': 0.16183519166000415, 'auc': 0.7501754385964912, 'training_loss': 33.25468449856474, 'f_test': 10.019414202217279, 'p': '***', 'y_pred': array([24.85847 , 43.477375, 38.90559 , 30.15303 , 46.29349 , 42.844856,
       39.900383, 22.341805, 26.407269, 32.23737 , 44.50212 , 43.703743,
       37.873283, 24.060806, 37.00432 , 40.333263, 40.67962 , 44.08265 ,
       38.17245 , 45.376717, 37.557327, 31.41845 , 38.57403 , 38.826954,
       31.503952, 48.409447, 37.541294, 22.717709, 34.26039 , 47.379906,
       38.898567, 42.42065 , 25.721447, 35.71694 , 32.897945, 38.11384 ,
       23.289528, 38.305447], dtype=float32), 'y_validate': array([13.314338 , 12.7686405, 45.95055  , 33.448204 , 39.851707 ,
       38.33021  , 32.91378  , 36.591187 , 12.776941 , 38.78881  ,
       39.108475 , 33.72665  , 41.073746 , 46.07022  , 43.956474 ,
       44.49892  , 35.705757 , 34.736805 , 44.761383 , 40.99727  ,
       36.716625 , 16.123117 , 34.9